# Panel Data

A **Panel** dataset is a dataset that incorporates both a time index and an "entity" index. For instance, imagine a study following students through their high school years. A unit of observation in this case would be a student/year combination.

This data format brings its own challenges!

Pandas is actually named afted this modelling (**Pa**nel **Da**ta**s**).

### Union Wage Data

The data set consists of wages and characteristics for men during the 1980s.  The entity identifier is ``nr`` and the time identified is ``year``.

Here a ``MultiIndex`` ``DataFrame`` is used to hold the data in a format that can be understood as a panel. Before setting the index, a year ``Categorical`` is created which facilitated making dummies.

In [ ]:
# !pip install linearmodels

In [5]:
from linearmodels.datasets import wage_panel
import pandas as pd

data = wage_panel.load()
year = pd.Categorical(data.year)
data = data.set_index(["nr", "year"])
data["year"] = year
print(wage_panel.DESCR)
data.sample(5)


F. Vella and M. Verbeek (1998), "Whose Wages Do Unions Raise? A Dynamic Model
of Unionism and Wage Rate Determination for Young Men," Journal of Applied
Econometrics 13, 163-183.

nr                       person identifier
year                     1980 to 1987
black                    =1 if black
exper                    labor market experience
hisp                     =1 if Hispanic
hours                    annual hours worked
married                  =1 if married
educ                     years of schooling
union                    =1 if in union
lwage                    log(wage)
expersq                  exper^2
occupation               Occupation code



,,black,exper,hisp,hours,married,educ,union,lwage,expersq,occupation,year
nr,year,,,,,,,,,,,
3621,1980,0,4,0,1860,0,12,0,1.508498,16,6,1980
3580,1983,0,5,0,2080,0,12,0,1.693661,25,4,1983
3202,1982,0,5,0,2116,1,12,1,1.680650,25,6,1982
1434,1985,0,7,0,2640,1,10,1,1.345229,49,7,1985
4258,1983,0,5,0,2056,0,12,1,2.140562,25,4,1983


## Basic regression on panel data

The simplest way to deal with panel data is to ignore it. Pooled Models simply work like a normal (cross section) model including a dummy variable for each $t$ time section.

In `linearmodels` we have ``PooledOLS`` is just plain OLS that understands that various panel data structures. It is useful as a base model. Here the log wage is modeled using all of the variables and time dummies.

In [15]:
from linearmodels.panel import PooledOLS
import statsmodels.api as sm

exog_vars = ["black", "hisp", "exper", "expersq", "married", "educ", "union", "year"]
exog = sm.add_constant(data[exog_vars])
mod = PooledOLS(data.lwage, exog)
pooled_res = mod.fit()
print(pooled_res)

PooledOLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                        0.1893
Estimator:                  PooledOLS   R-squared (Between):              0.2066
No. Observations:                4360   R-squared (Within):               0.1692
Date:                Sun, Apr 11 2021   R-squared (Overall):              0.1893
Time:                        22:51:05   Log-likelihood                   -2982.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      72.459
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(14,4345)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             72.459
                                        P-value       

We can also do it ourselves in pandas and statsmodels -- though we have to reset indices to make data useful:

In [16]:
df = data.reset_index(level=0).reset_index(drop=True)
df.head()

,nr,black,exper,hisp,hours,married,educ,union,lwage,expersq,occupation,year
0,13,0,1,0,2672,0,14,0,1.197540,1,9,1980
1,13,0,2,0,2320,0,14,1,1.853060,4,9,1981
2,13,0,3,0,2940,0,14,0,1.344462,9,9,1982
3,13,0,4,0,2960,0,14,0,1.433213,16,9,1983
4,13,0,5,0,3071,0,14,0,1.568125,25,5,1984


Then the model is the same in `OLS`:

In [28]:
X = df[['black', 'married', 'educ', 'hisp', 'union', 'expersq', 'exper']]
X  = X.join(pd.get_dummies(df.nr, drop_first=True))
X  = X.join(pd.get_dummies(df.year, drop_first=True))

sm.OLS(
    df.lwage,
    sm.add_constant(X)
).fit(cov_type='HC2').summary()

/Users/mranger/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:1834: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 558, but rank is 555
  'rank is %d' % (J, J_), ValueWarning)


0.567      -0.338       0.616
5274           0.6187      0.295      2.094      0.036       0.040       1.198
5335           0.0041      0.182      0.023      0.982      -0.352       0.360
5345           0.0960      0.209      0.459      0.647      -0.314       0.506
5359          -0.5736      0.127     -4.532      0.000      -0.822      -0.326
5368           0.1002      0.228      0.439      0.660      -0.347       0.547
5377           0.2456      0.195      1.257      0.209      -0.137       0.628
5390           0.1103      0.281      0.393      0.694      -0.440       0.661
5419           0.0085      0.187      0.046      0.964      -0.357       0.374
5435          -0.0834      0.142     -0.587      0.557      -0.362       0.195
5437           0.1281      0.189      0.677      0.498      -0.243       0.499
5497           0.0579      0.191      0.303      0.762      -0.316       0.432
5525          -0.2732      0.146     -1.876      0.061      -0.559       0.012
5529           0.1223      0.059      2.063      0.039       0.006       0.238
5531           0.2580      0.102      2.539      0.011       0.059       0.457
5579           0.1324      0.095      1.391      0.164      -0.054       0.319
5588          -0.2040      0.269     -0.759      0.448      -0.731       0.323
5599           0.3918      0.287      1.363      0.173      -0.171       0.955
5650           0.2129      0.145      1.465      0.143      -0.072       0.498
5660           0.5872      0.183      3.204      0.001       0.228       0.946
5665          -0.2000      0.254     -0.786      0.432      -0.699       0.299
5666           0.4949      0.319      1.552      0.121      -0.130       1.120
5698           0.1944      0.120      1.617      0.106      -0.041       0.430
5699           0.5214      0.235      2.220      0.026       0.061       0.982
5731          -0.1029      0.118     -0.870      0.384      -0.335       0.129
5750          -0.1151      0.137     -0.838      0.402      -0.384       0.154
5755          -0.0326      0.112     -0.291      0.771      -0.252       0.187
5772          -0.4321      0.323     -1.338      0.181      -1.065       0.201
5816          -0.2307      0.110     -2.104      0.035      -0.446      -0.016
5823          -0.3148      0.063     -5.004      0.000      -0.438      -0.191
5851          -0.0466      0.362     -0.129      0.897      -0.756       0.662
5857           0.1739      0.180      0.967      0.334      -0.179       0.526
5859           0.2295      0.354      0.649      0.516      -0.463       0.922
6016           0.5295      0.115      4.604      0.000       0.304       0.755
6020          -0.7513      0.148     -5.064      0.000      -1.042      -0.461
6025          -0.7171      0.217     -3.302      0.001      -1.143      -0.291
6056          -0.2594      0.139     -1.863      0.063      -0.532       0.014
6094          -0.0461      0.109     -0.422      0.673      -0.260       0.168
6186           0.3317      0.099      3.354      0.001       0.138       0.526
6395          -0.0018      0.278     -0.006      0.995      -0.547       0.543
6430          -0.2632      0.109     -2.406      0.016      -0.478      -0.049
6446           0.4387      0.139      3.163      0.002       0.167       0.711
6463          -0.5295      0.378     -1.400      0.162      -1.271       0.212
6558           0.6097      0.330      1.850      0.064      -0.036       1.256
6559           0.3830      0.330      1.161      0.246      -0.264       1.030
6561           0.7559      0.324      2.336      0.020       0.122       1.390
6574          -0.4119      0.094     -4.391      0.000      -0.596      -0.228
6648           0.6193      0.176      3.518      0.000       0.274       0.964
6813          -0.0584      0.052     -1.123      0.261      -0.160       0.044
6824          -0.1998      0.073     -2.738      0.006      -0.343      -0.057
6888          -0.6162      0.226     -2.726      0.006      -1.059      -0.173
6942           0.148

This means we can also throw any other valid $y ~ f(X)$ model at this format (lightgbm, deep learning, etc.)

## Estimating parameters with uncorrelated effects
When modeling panel data it is common to consider models beyond what OLS will efficiently estimate.  The most common are error component models which add an additional term to the standard OLS model,

$$ y_{it} = x_{it}\beta + \alpha_{i} + \epsilon_{it} $$

where $\alpha_i$ affects all values of entity i. When the $\alpha_i$ are uncorrelated with the regressors in $x_{it}$, a random effects model can be used to efficiently estimate parameters of this model.

### Random effects
The random effects model is virtually identical to the pooled OLS model except that is accounts for the structure of the model and so is more efficient. Random effects uses a quasi-demeaning strategy which subtracts the time average of the within entity values to account for the common shock.

In [23]:
from linearmodels.panel import RandomEffects

mod = RandomEffects(data.lwage, exog)
re_res = mod.fit()
print(re_res)

RandomEffects Estimation Summary                        
Dep. Variable:                  lwage   R-squared:                        0.1806
Estimator:              RandomEffects   R-squared (Between):              0.1853
No. Observations:                4360   R-squared (Within):               0.1799
Date:                Sun, Apr 11 2021   R-squared (Overall):              0.1828
Time:                        22:56:50   Log-likelihood                   -1622.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      68.409
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(14,4345)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             68.409
                                        P-value     

The model fit is fairly similar, although the return to experience has changed substantially, as has its significance.  This is partially explainable by the inclusion of the year dummies which will fit the trend in experience and so only the cross-sectional differences matter.  The quasi-differencing in the random effects estimator depends on a quantity that depends on the relative variance of the idiosyncratic shock and the common shock.  This can be accessed using ``variance_decomposition``.

# fixed effects

Entity effects are included by setting ``entity_effects=True``.  This is equivalent to including dummies for each entity.  In this panel, this would add 545 dummy variables and estimation of the model would be considerably slower.  ``PanelOLS`` does not actually use dummy variables and instead uses group-wise demeaning to achieve the same effect. 

#### Time-invariant Variables
Time-invariant variables cannot be included when using entity effects since, once demeaned, these will all be 0.  Here ``exper`` is also excluded since once entity effects and time dummies are incorporated, ``exper`` is perfectly co-linear.

In [27]:
from linearmodels.panel import PanelOLS

exog_vars = ["expersq", "union", "married", "year"]
exog = sm.add_constant(data[exog_vars])
mod = PanelOLS(data.lwage, exog, entity_effects=True)
fe_res = mod.fit()
print(fe_res)

PanelOLS Estimation Summary                           
Dep. Variable:                  lwage   R-squared:                        0.1806
Estimator:                   PanelOLS   R-squared (Between):             -0.0052
No. Observations:                4360   R-squared (Within):               0.1806
Date:                Sun, Apr 11 2021   R-squared (Overall):              0.0807
Time:                        23:02:10   Log-likelihood                   -1324.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      83.851
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(10,3805)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             83.851
                                        P-value       

Here's the equivalent in statsmodels with all the dummies estimated:

In [29]:
X = df[['black', 'married', 'educ', 'hisp', 'union', 'expersq', 'exper']]
X  = X.join(pd.get_dummies(df.nr, drop_first=True))
X  = X.join(pd.get_dummies(df.year, drop_first=True))

sm.OLS(
    df.lwage,
    sm.add_constant(X)
).fit(cov_type='HC2').summary()

/Users/mranger/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:1834: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 558, but rank is 555
  'rank is %d' % (J, J_), ValueWarning)


0.567      -0.338       0.616
5274           0.6187      0.295      2.094      0.036       0.040       1.198
5335           0.0041      0.182      0.023      0.982      -0.352       0.360
5345           0.0960      0.209      0.459      0.647      -0.314       0.506
5359          -0.5736      0.127     -4.532      0.000      -0.822      -0.326
5368           0.1002      0.228      0.439      0.660      -0.347       0.547
5377           0.2456      0.195      1.257      0.209      -0.137       0.628
5390           0.1103      0.281      0.393      0.694      -0.440       0.661
5419           0.0085      0.187      0.046      0.964      -0.357       0.374
5435          -0.0834      0.142     -0.587      0.557      -0.362       0.195
5437           0.1281      0.189      0.677      0.498      -0.243       0.499
5497           0.0579      0.191      0.303      0.762      -0.316       0.432
5525          -0.2732      0.146     -1.876      0.061      -0.559       0.012
5529           0.1223      0.059      2.063      0.039       0.006       0.238
5531           0.2580      0.102      2.539      0.011       0.059       0.457
5579           0.1324      0.095      1.391      0.164      -0.054       0.319
5588          -0.2040      0.269     -0.759      0.448      -0.731       0.323
5599           0.3918      0.287      1.363      0.173      -0.171       0.955
5650           0.2129      0.145      1.465      0.143      -0.072       0.498
5660           0.5872      0.183      3.204      0.001       0.228       0.946
5665          -0.2000      0.254     -0.786      0.432      -0.699       0.299
5666           0.4949      0.319      1.552      0.121      -0.130       1.120
5698           0.1944      0.120      1.617      0.106      -0.041       0.430
5699           0.5214      0.235      2.220      0.026       0.061       0.982
5731          -0.1029      0.118     -0.870      0.384      -0.335       0.129
5750          -0.1151      0.137     -0.838      0.402      -0.384       0.154
5755          -0.0326      0.112     -0.291      0.771      -0.252       0.187
5772          -0.4321      0.323     -1.338      0.181      -1.065       0.201
5816          -0.2307      0.110     -2.104      0.035      -0.446      -0.016
5823          -0.3148      0.063     -5.004      0.000      -0.438      -0.191
5851          -0.0466      0.362     -0.129      0.897      -0.756       0.662
5857           0.1739      0.180      0.967      0.334      -0.179       0.526
5859           0.2295      0.354      0.649      0.516      -0.463       0.922
6016           0.5295      0.115      4.604      0.000       0.304       0.755
6020          -0.7513      0.148     -5.064      0.000      -1.042      -0.461
6025          -0.7171      0.217     -3.302      0.001      -1.143      -0.291
6056          -0.2594      0.139     -1.863      0.063      -0.532       0.014
6094          -0.0461      0.109     -0.422      0.673      -0.260       0.168
6186           0.3317      0.099      3.354      0.001       0.138       0.526
6395          -0.0018      0.278     -0.006      0.995      -0.547       0.543
6430          -0.2632      0.109     -2.406      0.016      -0.478      -0.049
6446           0.4387      0.139      3.163      0.002       0.167       0.711
6463          -0.5295      0.378     -1.400      0.162      -1.271       0.212
6558           0.6097      0.330      1.850      0.064      -0.036       1.256
6559           0.3830      0.330      1.161      0.246      -0.264       1.030
6561           0.7559      0.324      2.336      0.020       0.122       1.390
6574          -0.4119      0.094     -4.391      0.000      -0.596      -0.228
6648           0.6193      0.176      3.518      0.000       0.274       0.964
6813          -0.0584      0.052     -1.123      0.261      -0.160       0.044
6824          -0.1998      0.073     -2.738      0.006      -0.343      -0.057
6888          -0.6162      0.226     -2.726      0.006      -1.059      -0.173
6942           0.148

You may notice that fitting all of these dummy variables is slow.

It also **artificially biases the R2** (even the adjusted R2)!

The common way to bypass this (that `linearmodels` uses) is to remove the mean of $y$ at the individual group level.

Note that this is mathematically equivalent to fitting a dummy variable (since the variable is the mean where the dummy is $=1$)

In [34]:
X = df[['black', 'married', 'educ', 'hisp', 'union', 'expersq', 'exper']]
X  = X.join(pd.get_dummies(df.year, drop_first=True))

y = df.lwage.copy()
# Demean y from nr group averages
y -= df.groupby('nr').lwage.transform('mean')

sm.OLS(
    y,
    sm.add_constant(X)
).fit(cov_type='HC2').summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  lwage   R-squared:                       0.173
Model:                            OLS   Adj. R-squared:                  0.171
Method:                 Least Squares   F-statistic:                     56.30
Date:                Sun, 11 Apr 2021   Prob (F-statistic):          1.02e-145
Time:                        23:09:44   Log-Likelihood:                -1343.7
No. Observations:                4360   AIC:                             2717.
Df Residuals:                    4345   BIC:                             2813.
Df Model:                          14                                         
Covariance Type:                  HC2                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.3498      0.054     -6.444      0.000      -0.456      -0.243
black         -0.0032      0.017     -0.191      0.849      -0.036       0.030
married        0.0176      0.010      1.718      0.086      -0.002       0.038
educ          -0.0034      0.004     -0.960      0.337      -0.010       0.004
hisp           0.0021      0.014      0.148      0.882      -0.025       0.029
union          0.0282      0.011      2.557      0.011       0.007       0.050
expersq       -0.0037      0.001     -6.688      0.000      -0.005      -0.003
exper          0.0557      0.010      5.782      0.000       0.037       0.075
1981           0.0882      0.025      3.532      0.000       0.039       0.137
1982           0.1237      0.025      4.855      0.000       0.074       0.174
1983           0.1556      0.026      5.888      0.000       0.104       0.207
1984           0.2187      0.029      7.500      0.000       0.162       0.276
1985           0.2684      0.031      8.629      0.000       0.207       0.329
1986           0.3365      0.034     10.001      0.000       0.271       0.402
1987           0.4167      0.035     12.029      0.000       0.349       0.485
==============================================================================
Omnibus:                     2340.921   Durbin-Watson:                   1.909
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            50353.284
Skew:                          -2.095   Prob(JB):                         0.00
Kurtosis:                      19.113   Cond. No.                         929.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC2)
"""

# first differences
First differencing is an alternative to using fixed effects when there might be correlation.  When using first differences, time-invariant variables must be excluded.  Additionally, only one linear time-trending variable can be included since this will look like a constant.  This variable will soak up all time-trends in the data, and so interpretations of these variable can be challenging.

In [35]:
from linearmodels.panel import FirstDifferenceOLS

exog_vars = ["exper", "expersq", "union", "married"]
exog = data[exog_vars]
mod = FirstDifferenceOLS(data.lwage, exog)
fd_res = mod.fit()
print(fd_res)

FirstDifferenceOLS Estimation Summary                      
Dep. Variable:                  lwage   R-squared:                        0.0268
Estimator:         FirstDifferenceOLS   R-squared (Between):              0.5491
No. Observations:                3815   R-squared (Within):               0.1763
Date:                Sun, Apr 11 2021   R-squared (Overall):              0.5328
Time:                        23:12:28   Log-likelihood                   -2305.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      26.208
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                  F(4,3811)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             26.208
                                        P-value  

# Unbalanced Panels

If a panel is missing some individuals at some times (eg. it's **not complete**) we call it "unbalanced". 

This is a real annoyance, unless you can assume the data is **MCAR** (missing completely at random).

Otherwise, if your data is missing non-randomly, then your estimate of the $y$ variable is biased (it's missing when it should be observed in a way correlated with the dataset). This problem doesn't go away if you use a GBDT or DeepNet -- it's a fundamental property of the dataset.

The only way to fix this is to do **imputation** (if it makes sense) or to **model in the missing data** (using something like the 2-stage Hurdle Model we saw at the end of module 3).

# Dynamic Panels

Mixing autoregressive (AR) terms and panel models is a tricky beast we won't dive into deeply in this course.

You can include lags on $Y$ (called a **dynamic panel**) and lags on $X$ (Called **Lagged Dependent Variable**) if it makes theoretical sense for you.

Both of these can help the model but can also hurt in other ways, or introduce bias. Buyer beware.